<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/VIsualBERT_VQA_COCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VIsualBERT VQA COCO
paper: https://arxiv.org/pdf/1908.03557.pdf <br>
We use the VQA
2.0 (Goyal et al., 2017), consisting of over 1 million questions about images from COCO. We train
the model to predict the 3,129 most frequent answers and use image features from a ResNeXt-based
Faster RCNN pre-trained on Visual Genome (Jiang et al., 2018).

For COCO pre-training, first download COCO caption annotations to a folder X_COCO.

In [1]:
!mkdir X_COCO
%cd X_COCO
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip -q annotations_trainval2014.zip

/content/X_COCO
--2023-02-05 01:47:50--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.220.193, 54.231.199.41, 52.217.229.137, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.220.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M   101MB/s    in 2.4s    

2023-02-05 01:47:53 (101 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]



Then download COCO image features to X_COCO.<br> 
Train: https://drive.google.com/file/d/1F-LSQhpKleV4nmiKMjQvpHMS2gkbK3bY/view?usp=sharing <br> 
Val: https://drive.google.com/file/d/1cZjPob3YqfM46LaWY3-Ky12claxeXbWi/view?usp=sharing

In [2]:
# Train: https://drive.google.com/file/d/1F-LSQhpKleV4nmiKMjQvpHMS2gkbK3bY/view?usp=sharing
# Val: https://drive.google.com/file/d/1cZjPob3YqfM46LaWY3-Ky12claxeXbWi/view?usp=sharing

#Option1:
# import gdown
# url = 'https://drive.google.com/uc?id=1F-LSQhpKleV4nmiKMjQvpHMS2gkbK3bY'
# gdown.download(url,'coco_features_train_150.th ',quiet=True) 

#Option2:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# id = ['1F-LSQhpKleV4nmiKMjQvpHMS2gkbK3bY']
# downloaded = drive.CreateFile({'id':id[0]}) 
# downloaded.GetContentFile('coco_features_train_150.th')

#Option3:
!curl -L -s -o coco_features_train_150.th 'https://drive.google.com/uc?id=1F-LSQhpKleV4nmiKMjQvpHMS2gkbK3bY'
!curl -L -s -o coco_features_val_150.th 'https://drive.google.com/uc?id=1cZjPob3YqfM46LaWY3-Ky12claxeXbWi'

Prepare VQA Data
The image features and VQA data are from Pythia. Assuming the data is stored in X_COCO.



In [7]:
!mkdir data
%cd data
!wget https://dl.fbaipublicfiles.com/pythia/data/vocabulary_vqa.txt
!wget https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt
!wget https://dl.fbaipublicfiles.com/pythia/data/imdb.tar.gz
!gunzip -q imdb.tar.gz 
!tar -xf imdb.tar

!wget https://dl.fbaipublicfiles.com/pythia/features/detectron_fix_100.tar.gz
!gunzip -q detectron_fix_100.tar.gz
!tar -xf detectron_fix_100.tar
!rm -f detectron_fix_100.tar

/content/X_COCO/data
--2023-02-05 02:02:23--  https://dl.fbaipublicfiles.com/pythia/data/vocabulary_vqa.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142136 (139K) [text/plain]
Saving to: ‘vocabulary_vqa.txt’

vocabulary_vqa.txt  100%[===================>] 138.80K   580KB/s    in 0.2s    

2023-02-05 02:02:24 (580 KB/s) - ‘vocabulary_vqa.txt’ saved [142136/142136]

--2023-02-05 02:02:24--  https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24768 (24K) [text/plain]
Saving to: ‘answers_vqa.txt’

answers_vqa.txt     1

Download raw COCO images:

In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!unzip -q train2014.zip
!unzip -q val2014.zip

COCO VQA Datalaoder: https://github.com/uclanlp/visualbert/blob/master/visualbert/dataloaders/vqa_dataset.py <br>
another1: https://github.com/Axe--/Visual-Question-Answering/blob/master/dataloader.py